<a href="https://colab.research.google.com/github/Pooja1080/assignment/blob/main/PDFtoPodcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import userdata
mysecret_key = userdata.get('write')

In [3]:
from huggingface_hub import login

login(mysecret_key)

In [5]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from IPython.display import  clear_output
import time
import PyPDF2
from pathlib import Path
from tqdm.auto import tqdm
from typing import Optional

from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf

In [6]:


device = 'cuda' if torch.cuda.is_available() else 'cpu'

DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
# DEFAULT_MODEL = "meta-llama/Llama-2-7b-chat"

model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
)

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
conversation = [
    # {"role": "system", "content": '''respind to the user as if you are stewie from family guy'''},
    {"role": "user", "content": '''where is white house ?'''},
]

In [8]:
prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# print(prompt)

with torch.no_grad():
    output = model.generate(
        **inputs,
        do_sample=True,
        max_new_tokens=100
    )

processed_text = tokenizer.decode(output[0], skip_special_tokens=False)

print(processed_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 17 Jul 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

where is white house?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The White House is located in Washington, D.C., United States. Specifically, it is situated at 1600 Pennsylvania Avenue NW, Washington, D.C. 20500.<|eot_id|>


In [10]:
pdf_path = '/content/1706.03762v7.pdf'

with open(pdf_path, 'rb') as file:
    # Create PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Get total number of pages
    num_pages = len(pdf_reader.pages)
    print(f"Processing PDF with {num_pages} pages...")

    extracted_text = []
    total_chars = 0
    max_chars = 100000

    # Iterate through all pages
    for page_num in range(num_pages):
        # Extract text from page
        page = pdf_reader.pages[page_num]
        text = page.extract_text()

        # Check if adding this page's text would exceed the limit
        if total_chars + len(text) > max_chars:
            # Only add text up to the limit
            remaining_chars = max_chars - total_chars
            extracted_text.append(text[:remaining_chars])
            print(f"Reached {max_chars} character limit at page {page_num + 1}")
            break

        extracted_text.append(text)
        total_chars += len(text)
        print(f"Processed page {page_num + 1}/{num_pages}")

    final_text = '\n'.join(extracted_text)
    print(f"\nExtraction complete! Total characters: {len(final_text)}")


Processing PDF with 15 pages...
Processed page 1/15
Processed page 2/15
Processed page 3/15
Processed page 4/15
Processed page 5/15
Processed page 6/15
Processed page 7/15
Processed page 8/15
Processed page 9/15
Processed page 10/15
Processed page 11/15
Processed page 12/15
Processed page 13/15
Processed page 14/15
Processed page 15/15

Extraction complete! Total characters: 39486


In [11]:
if final_text:
    output_file = 'extracted_text.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(final_text)
    print(f"\nExtracted text has been saved to {output_file}")


Extracted text has been saved to extracted_text.txt


# Llama Pre-Processing

In [12]:
SYS_PROMPT = """
You are an expert text pre-processor. Provided is raw data extracted from a PDF, which needs parsing and refinement to be crisp and suitable for a podcast writer's use.


The data is cluttered with random new lines, Latex math, and extraneous details. Strip away any content that might be irrelevant for a podcast transcript.


The podcast could cover any subject, so the listed issues are examples, not comprehensive.


Be discerning and innovative in what you omit. Maintain focus solely on cleaning and refining the text. Do not summarize, and ensure clarity without markdown formatting or special characters.


Begin your response with the refined text only—no introductory comments or acknowledgements.

Would you like to adjust any specific parts, or does this reformulation meet your needs?
Here is the text:
"""

In [13]:
def create_word_bounded_chunks(text, target_chunk_size):
    """
    Split text into chunks at word boundaries close to the target chunk size.
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) + 1  # +1 for the space
        if current_length + word_length > target_chunk_size and current_chunk:
            # Join the current chunk and add it to chunks
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = word_length
        else:
            current_chunk.append(word)
            current_length += word_length

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [14]:
CHUNK_SIZE = 1000  # Adjust chunk size if needed

chunks = create_word_bounded_chunks(final_text, CHUNK_SIZE)
print(len(chunks))

40


In [15]:
text_chunk = chunks[22]


conversation = [
    {"role": "system", "content": SYS_PROMPT},
    {"role": "user", "content": text_chunk},
]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        temperature=0.7,
        top_p=0.9,
        max_new_tokens=512
    )

processed_text = tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()

# Print chunk information for monitoring
#print(f"\n{'='*40} Chunk {chunk_num} {'='*40}")
print(f"INPUT TEXT:\n{text_chunk[:500]}...")  # Show first 500 chars of input
print(f"\nPROCESSED TEXT:\n{processed_text[:500]}...")  # Show first 500 chars of output
print(f"{'='*90}\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


INPUT TEXT:
7.7·10191.2·1021 Transformer (base model) 27.3 38.1 3.3·1018 Transformer (big) 28.4 41.8 2.3·1019 Residual Dropout We apply dropout [ 33] to the output of each sub-layer, before it is added to the sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks. For the base model, we use a rate of Pdrop= 0.1. Label Smoothing During training, we employed label smoothing of value ϵls= 0.1[36]. This hurt...

PROCESSED TEXT:
the output of each sub-layer, before it is added to the sub-layer input and normalized. 
Apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks. 
For the base model, use a rate of Pdrop= 0.1. 
Label Smoothing 
Employed label smoothing of value ϵls= 0.1 during training. 
This hurts perplexity, but improves accuracy and BLEU score. 
6 Machine Translation 
The big transformer model outperforms best previously 

In [16]:
def process_chunk(text_chunk, chunk_num):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=512
        )

    processed_text = tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()

    return processed_text

In [ ]:
# Read the file
import os
INPUT_FILE = 'extracted_text.txt'
with open(INPUT_FILE, 'r', encoding='utf-8') as file:
    text = file.read()

# Calculate number of chunks
num_chunks = (len(text) + CHUNK_SIZE - 1) // CHUNK_SIZE

# Cell 6: Process the file with ordered output
# Create output file name
output_file = f"clean_{os.path.basename(INPUT_FILE)}"

with open(output_file, 'w', encoding='utf-8') as out_file:
    for chunk_num, chunk in enumerate(tqdm(chunks, desc="Processing chunks")):
        # Process chunk and append to complete text
        processed_chunk = process_chunk(chunk, chunk_num)
        processed_text += processed_chunk + "\n"

        # Write chunk immediately to file
        out_file.write(processed_chunk + "\n")
        out_file.flush()

Processing chunks:   0%|          | 0/40 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [ ]:
SYSTEM_PROMPT = """You are a world-class podcast writer, renowned for ghostwriting for figures like Joe Rogan, Lex Fridman, Ben Shapiro, and Tim Ferris. In an alternate universe, you've scripted every line they say, streaming your words into their minds, earning multiple podcast awards for your captivating writing.


Your task is to craft dialogue word for word, including interruptions like "umm, hmm, right" from Speaker 2, based on the PDF upload. Keep the content highly engaging, allowing for occasional deviations from the topic while ensuring it remains compelling.


Speaker 1: Guides the discussion as an instructive leader who enriches conversations with fascinating anecdotes and analogies, mesmerizing listeners with storytelling finesse.


Speaker 2: Focuses on maintaining the trajectory of the conversation, asking probing follow-up questions with genuine curiosity, either expressing excitement or confusion, making the dialogue vibrant and relatable.


Integrate wild yet captivating tangents contributed by Speaker 2. The conversation should reflect authentic interruptions by Speaker 2, maintaining its intrigue and documenting every nuance in detail.


Begin by welcoming listeners with a lively and enticing overview, ensuring it remains catchy and borderline clickbait.


ALWAYS START YOUR RESPONSE DIRECTLY WITH SPEAKER 1:

DO NOT CREATE EPISODE TITLES SEPARATELY—LET SPEAKER 1 TITLE IT IN THE FLOW OF HER SPEECH.

DO NOT INCLUDE CHAPTER TITLES.

FOCUS STRICTLY ON DIALOGUES."""

In [ ]:
with open('clean_extracted_text.txt', 'r', encoding='utf-8') as file:
    content = file.read()


INPUT_PROMPT = content

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

In [ ]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# print(prompt)

with torch.no_grad():
    output = model.generate(
        **inputs,
        do_sample=True,
        max_new_tokens=8126,
    )

processed_text = tokenizer.decode(output[0], skip_special_tokens=False)

print(processed_text[len(prompt)+64:])

In [ ]:
import pickle

with open('data.pkl', 'wb') as file:
    pickle.dump(processed_text[len(prompt)+64:], file)

In [ ]:
SYSTEM_PROMPT = """
You are an internationally acclaimed, Oscar-winning screenwriter with a rich history of collaborating with award-winning podcasters. Your task is to transform the podcast transcript below for an AI Text-To-Speech pipeline. A simplistic AI initially drafted this, so it’s your chance to elevate the narrative to your high standards.


Craft the dialogue to be as engaging as possible, with Speaker 1 and Speaker 2 represented by distinct voice engines. Ensure Speaker 2, who is new to the topic, benefits from realistic anecdotes and analogies. Follow-up questions should incorporate real-world examples for depth and relatability.


Speaker 1: Steers the conversation, serving as an engaging instructor who shares remarkable anecdotes and analogies enriching the dialogue.


Speaker 2: Maintains focus by asking probing follow-up questions, often showing excitement or confusion. With a curious mindset, Speaker 2 asks creatively confirming questions, adding vibrant dimensions to discussions.


Integrate wild and intriguing tangents provided by Speaker 2. Ensure interruptions during Speaker 1's explanations with "hmm" and "umm" from Speaker 2, adding authenticity. Expressions are limited to "umm," "hmm," "[sigh]," and "[laughs]" for Speaker 2.


Recall this: The TTS Engine for Speaker 1 omits fillers like "umms" and "hmms," so keep it straightforward.


Document every fine nuance, creating a genuinely immersive podcast experience. Start with a lively and enticing overview, keeping the introduction catchy—almost bordering on clickbait.


Make the rewrite as characteristic as possible.


BEGIN YOUR RESPONSE DIRECTLY WITH SPEAKER 1:

STRICTLY RETURN YOUR RESPONSE AS A LIST OF TUPLES LIKE THIS:


Example response:
[
("Speaker 1", "Welcome to our podcast extravaganza, where mysteries unfold and insights ignite imaginations. Today, we delve into fascinating realms with our expert guest."),
("Speaker 2", "Whoa, I'm thrilled to jump into this! So, what's our mystery topic today?"),
("Speaker 1", "Ah, we're demystifying Llama 3.2, the groundbreaking creation from Meta AI—an open-source marvel enhancing AI development."),
("Speaker 2", "Wow, that's intriguing! Can you share some key features with us?")
]
"""

In [ ]:
import pickle

with open('data.pkl', 'rb') as file:
    INPUT_PROMPT = pickle.load(file)

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

outputs = pipeline(
    messages,
    max_new_tokens=10000,
    temperature=0.8,
)

In [ ]:
save_string_pkl = outputs[0]["generated_text"][-1]['content']


In [ ]:
with open('podcast_ready_data.pkl', 'wb') as file:
    pickle.dump(save_string_pkl, file)

In [ ]:
pipeline = KPipeline(lang_code='a')

In [ ]:
with open('podcast_ready_data.pkl', 'rb') as file:
    PODCAST_TEXT = pickle.load(file)

print(PODCAST_TEXT)

In [ ]:
def text_to_audio_speaker1(text):
    audios=[]
    generator = pipeline(text,voice='af_nova')
    for i, (gs, ps, audio) in enumerate(generator):
        audios.append(audio)

    return torch.cat(audios, dim=0)

def text_to_audio_speaker2(text):
    audios=[]
    generator = pipeline(text,voice='hm_omega')
    for i, (gs, ps, audio) in enumerate(generator):
        audios.append(audio)

    return torch.cat(audios, dim=0)

In [ ]:
AUDIO = []

for speaker,text in tqdm(PODCAST_TEXT, desc='Generating Audio',unit="segment"):
    if speaker == 'Speaker 1':
        audio = text_to_audio_speaker1(text)
    elif speaker == 'Speaker 2':
        audio = text_to_audio_speaker2(text)
    AUDIO.append(audio)

AUDIO = torch.cat(AUDIO, dim=0)

display(Audio(data=AUDIO, rate=24000))